In [16]:
import pandas as pd 
import sqlite3 


In [17]:
pip install ipython-sql

Note: you may need to restart the kernel to use updated packages.


In [18]:
path = "D:/project/"  #Insert path here
db = path + 'database.sqlite'
conn = sqlite3.connect(db)
cur = conn.cursor()

In [19]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [20]:
%sql sqlite:///D:/project/database.sqlite 

In [21]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///D:/project/database.sqlite
Done.


name
sqlite_sequence
Player_Attributes
Player
Match
League
Country
Team
Team_Attributes


### 1. list of League from country

In [22]:
%%sql SELECT c.name AS country_name, l.name AS leage_name FROM Country c
JOIN League l
ON c.id = l.country_id;

 * sqlite:///D:/project/database.sqlite
Done.


country_name,leage_name
Belgium,Belgium Jupiler League
England,England Premier League
France,France Ligue 1
Germany,Germany 1. Bundesliga
Italy,Italy Serie A
Netherlands,Netherlands Eredivisie
Poland,Poland Ekstraklasa
Portugal,Portugal Liga ZON Sagres
Scotland,Scotland Premier League
Spain,Spain LIGA BBVA


### 2.จำจวนทีมในแต่ละลีค

In [23]:
%%sql
SELECT l.name AS league_name, COUNT(DISTINCT t.team_api_id) AS team_count
FROM League l
JOIN Match m ON l.id = m.league_id
JOIN Team t ON m.home_team_api_id = t.team_api_id
GROUP BY l.name
ORDER BY team_count DESC;

 * sqlite:///D:/project/database.sqlite
Done.


league_name,team_count
France Ligue 1,35
England Premier League,34
Spain LIGA BBVA,33
Italy Serie A,32
Germany 1. Bundesliga,30
Portugal Liga ZON Sagres,29
Netherlands Eredivisie,25
Belgium Jupiler League,25
Poland Ekstraklasa,24
Scotland Premier League,17


### 3. over all ของนักเตะที่สูงที่สุด 10 อันดับ

In [24]:
%%sql
SELECT p.player_name, MAX(pa.overall_rating)AS overall_rating FROM Player_Attributes pa,Player p
WHERE pa.player_api_id = p.player_api_id
GROUP BY p.player_name
ORDER BY overall_rating DESC LIMIT 10;

 * sqlite:///D:/project/database.sqlite
Done.


player_name,overall_rating
Lionel Messi,94
Wayne Rooney,93
Gianluigi Buffon,93
Cristiano Ronaldo,93
Xavi Hernandez,92
Gregory Coupet,92
Thierry Henry,91
Ronaldinho,91
John Terry,91
Iker Casillas,91


### 4. จำนวนทีมที่ชนะมากที่สุดในทุกฤดูกาล

In [49]:
%%sql
SELECT t.team_long_name, COUNT(DISTINCT m.match_api_id) AS total_win
FROM Match m
JOIN Team t 
ON t.team_api_id = m.home_team_api_id OR t.team_api_id = m.away_team_api_id
WHERE 
(m.home_team_goal > m.away_team_goal AND t.team_api_id = m.home_team_api_id) OR
(m.away_team_goal > m.home_team_goal AND t.team_api_id = m.away_team_api_id)
GROUP BY 
t.team_long_name
ORDER BY 
total_win DESC
LIMIT 10;

 * sqlite:///D:/project/database.sqlite
Done.


team_long_name,total_win
FC Barcelona,234
Real Madrid CF,228
Celtic,218
FC Bayern Munich,193
Manchester United,192
Juventus,189
SL Benfica,185
FC Porto,183
Ajax,181
FC Basel,180


### 5. ทีมที่ชนะมากที่สุดในฤดูกาล 2015/2016

In [50]:
%%sql
SELECT t.team_long_name, COUNT(DISTINCT m.match_api_id) AS total_wins
FROM Match m
JOIN Team t 
ON t.team_api_id = m.home_team_api_id OR t.team_api_id = m.away_team_api_id
WHERE 
((m.home_team_goal > m.away_team_goal AND t.team_api_id = m.home_team_api_id) OR
(m.away_team_goal > m.home_team_goal AND t.team_api_id = m.away_team_api_id))
AND m.season = '2015/2016'
GROUP BY 
t.team_long_name
ORDER BY 
total_wins DESC
LIMIT 10;

 * sqlite:///D:/project/database.sqlite
Done.


team_long_name,total_wins
Paris Saint-Germain,30
SL Benfica,29
Juventus,29
FC Barcelona,29
Real Madrid CF,28
FC Bayern Munich,28
Atlético Madrid,28
Sporting CP,27
PSV,26
FC Basel,26


### 6. ชนะรวมทั้งหมดจำแนกตามซีซั่น ของแมนยู

In [51]:
%%sql
SELECT m.season,t.team_long_name, COUNT(DISTINCT m.match_api_id) AS total_wins
FROM Match m
JOIN Team t 
ON t.team_api_id = m.home_team_api_id OR t.team_api_id = m.away_team_api_id
WHERE 
((m.home_team_goal > m.away_team_goal AND t.team_api_id = m.home_team_api_id) OR
(m.away_team_goal > m.home_team_goal AND t.team_api_id = m.away_team_api_id))
AND t.team_long_name = 'Manchester United'
GROUP BY t.team_long_name, m.season;

 * sqlite:///D:/project/database.sqlite
Done.


season,team_long_name,total_wins
2008/2009,Manchester United,28
2009/2010,Manchester United,27
2010/2011,Manchester United,23
2011/2012,Manchester United,28
2012/2013,Manchester United,28
2013/2014,Manchester United,19
2014/2015,Manchester United,20
2015/2016,Manchester United,19


### 7. แมนยูชนะในบ้านจำแนกตามฤดูกาล

In [53]:
%%sql

SELECT m.season, t.team_long_name, COUNT(m.home_team_api_id) AS home_win
FROM Team t
JOIN Match m ON t.team_api_id = m.home_team_api_id
WHERE m.home_team_goal > m.away_team_goal AND t.team_long_name = 'Manchester United'
GROUP BY m.season, t.team_long_name;


 * sqlite:///D:/project/database.sqlite
Done.


season,team_long_name,home_win
2008/2009,Manchester United,16
2009/2010,Manchester United,16
2010/2011,Manchester United,18
2011/2012,Manchester United,15
2012/2013,Manchester United,16
2013/2014,Manchester United,9
2014/2015,Manchester United,14
2015/2016,Manchester United,12


### 8. แมนยูชนะนอกบ้านจำแนกตามฤดูกาล

In [29]:
%%sql

SELECT m.season, t.team_long_name, COUNT(m.away_team_api_id) AS away_win
FROM Team t
JOIN Match m ON t.team_api_id = m.away_team_api_id
WHERE m.home_team_goal < m.away_team_goal AND t.team_long_name = 'Manchester United'
GROUP BY m.season, t.team_long_name;

 * sqlite:///D:/project/database.sqlite
Done.


season,team_long_name,away_win
2008/2009,Manchester United,12
2009/2010,Manchester United,11
2010/2011,Manchester United,5
2011/2012,Manchester United,13
2012/2013,Manchester United,12
2013/2014,Manchester United,10
2014/2015,Manchester United,6
2015/2016,Manchester United,7


### 9. ข้อมูลการยิงประตูของแมนยู

In [56]:
%%sql

SELECT m.season, t.team_long_name, SUM(m.home_team_goal) AS home_goals, SUM(m.away_team_goal) AS away_goals,
SUM(m.home_team_goal + m.away_team_goal) AS total_goals, ROUND(AVG(m.home_team_goal + m.away_team_goal), 1) AS AVG_goals
FROM  Match m
JOIN Team t 
ON t.team_api_id = m.home_team_api_id OR t.team_api_id = m.away_team_api_id
WHERE 
t.team_long_name = 'Manchester United'
GROUP BY 
t.team_long_name, m.season;


 * sqlite:///D:/project/database.sqlite
Done.


season,team_long_name,home_goals,away_goals,total_goals,AVG_goals
2008/2009,Manchester United,54,38,92,2.4
2009/2010,Manchester United,68,46,114,3.0
2010/2011,Manchester United,74,41,115,3.0
2011/2012,Manchester United,66,56,122,3.2
2012/2013,Manchester United,69,60,129,3.4
2013/2014,Manchester United,51,56,107,2.8
2014/2015,Manchester United,63,36,99,2.6
2015/2016,Manchester United,53,31,84,2.2


### 10. ทีมที่แพ้แมนยูมากที่สุด 5 อันดับ

In [61]:
%%sql
SELECT t.team_long_name AS team, COUNT(t.team_api_id) AS total_losses
FROM Team t
JOIN Match m
ON (t.team_api_id = m.home_team_api_id 
    AND m.away_team_api_id = (SELECT team_api_id FROM Team WHERE team_long_name = 'Manchester United'))
OR (t.team_api_id = m.away_team_api_id
    AND m.home_team_api_id = (SELECT team_api_id FROM Team WHERE team_long_name = 'Manchester United'))
WHERE
    ((m.home_team_goal < m.away_team_goal AND t.team_api_id = m.home_team_api_id)
    OR (m.away_team_goal < m.home_team_goal AND t.team_api_id = m.away_team_api_id))
    AND m.league_id = 1729
GROUP BY
    t.team_long_name
ORDER BY
    total_losses DESC
LIMIT 5;

 * sqlite:///D:/project/database.sqlite
Done.


team,total_losses
Stoke City,12
Sunderland,11
Aston Villa,11
West Ham United,10
Wigan Athletic,9
